In [388]:
import os
import math
import stat
import pandas as pd
import numpy as np
import re
from sklearn import preprocessing
import tensorflow as tf

In [389]:
os.getcwd()

'/Users/dominikpeter/Homework/mining/Tensorflow'

In [390]:
x = [x for x in os.walk('/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset')]

In [391]:
subfolder = x[0][1]
subfolder[0:5]

['Brush_teeth',
 'Climb_stairs',
 'Climb_stairs_MODEL',
 'Comb_hair',
 'Descend_stairs']

In [392]:
folders = []
[folders.append('/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset' + '/' + word) for word in subfolder]
folders[0:5]

['/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Climb_stairs',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Climb_stairs_MODEL',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Comb_hair',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Descend_stairs']

In [393]:
x = ([x[0] for x in os.walk('/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth')])

In [394]:
files = []
for folder in folders:
    [files.append(folder + '/' + f) for f in listdir(folder) if f.endswith(".txt")]
files[0:5]

['/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-28-18-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-04-11-13-29-54-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-08-35-11-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-09-36-50-brush_teeth-f1.txt',
 '/Users/dominikpeter/Homework/mining/Tensorflow/HMP_Dataset/Brush_teeth/Accelerometer-2011-05-30-10-34-16-brush_teeth-m1.txt']

In [395]:
df = []
for file in files:
    new_df = pd.read_csv(file, sep = ' ', header=None, names = ['x', 'y', 'z'])
    new_df['label'] = re.search('-\w+-', file[::-1]).group(0)[::-1].replace('-', '')
    df.append(new_df)
df = pd.concat(df, axis=0)
df.head()

,x,y,z,label
0,22,49,35,brush_teeth
1,22,49,35,brush_teeth
2,22,52,35,brush_teeth
3,22,52,35,brush_teeth
4,21,52,34,brush_teeth


In [396]:
df.shape

(479289, 4)

In [397]:
df = df.join(df.groupby('label')[['x', 'y','z']].mean(), on='label', rsuffix='.mean')
df.head()

,x,y,z,label,x.mean,y.mean,z.mean
0,22,49,35,brush_teeth,27.654397,49.281505,34.500654
1,22,49,35,brush_teeth,27.654397,49.281505,34.500654
2,22,52,35,brush_teeth,27.654397,49.281505,34.500654
3,22,52,35,brush_teeth,27.654397,49.281505,34.500654
4,21,52,34,brush_teeth,27.654397,49.281505,34.500654


In [398]:
df['diff.x'] = abs(df['x'] - df['x.mean'])
df['diff.y'] = abs(df['y'] - df['y.mean'])
df['diff.z'] = abs(df['z'] - df['z.mean'])

In [399]:
df.head()

,x,y,z,label,x.mean,y.mean,z.mean,diff.x,diff.y,diff.z
0,22,49,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,0.281505,0.499346
1,22,49,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,0.281505,0.499346
2,22,52,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,2.718495,0.499346
3,22,52,35,brush_teeth,27.654397,49.281505,34.500654,5.654397,2.718495,0.499346
4,21,52,34,brush_teeth,27.654397,49.281505,34.500654,6.654397,2.718495,0.500654


In [400]:
le = preprocessing.LabelEncoder()
le.fit(df['label'])
le.classes_

array(['brush_teeth', 'climb_stairs', 'comb_hair', 'descend_stairs',
       'drink_glass', 'eat_meat', 'eat_soup', 'getup_bed', 'liedown_bed',
       'pour_water', 'sitdown_chair', 'standup_chair', 'use_telephone',
       'walk'], dtype=object)

In [401]:
df['class'] = le.transform(df['label'])

In [403]:
y = pd.get_dummies(df['class'])
y.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [275]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))


def model(X, w):
    return tf.matmul(X, w)

In [299]:
X = df[['x', 'y', 'z']]

,x,y,z
0,22,49,35
1,22,49,35
2,22,52,35
3,22,52,35
4,21,52,34
5,22,51,34
6,20,50,35
7,22,52,34
8,22,50,34
9,22,51,35


In [276]:
X = tf.placeholder("float", [None, 784])
Y = tf.placeholder("float", [None, 10])

w_h = init_weights([784, 625]) # create symbolic variables
w_o = init_weights([625, 10])

In [277]:
py_x = model(X, w_h, w_o)

TypeError: model() takes 2 positional arguments but 3 were given